In [14]:
import pymysql


# connection 을 열고, cursor 열고, 입력받은 query 를 실행한 뒤에, 얻어온 정보를 result 에 저장, 연결들을 모두 해제한 뒤, result 를 다시 반환.
def wrapper_sql(query):
    conn = pymysql.connect(
    host ='localhost',
    port = 3306,
    user = 'movie',
    passwd = '1234',
    db = 'movie')
    # with말고 try except를 사용하기
    # with conn:
    #     with conn.cursor() as cursor:
    #         cursor.execute(query)
    #         result = cursor.fetchall()
    #         conn.commit()
    # return result
    try:
        cursor = conn.cursor()
        cursor.execute(query)    
        result = cursor.fetchall()
    except:
        print('query: {}'.format(query))
        print('잘못된 SQL 쿼리입니다!!!')
        result = -2
    finally:
        conn.commit()
        cursor.close()
        return result

    


def insert_movie_info():
    print('영화 정보를 입력할 수 있습니다.')
    info_category = input('1) 영화 2) 배우 3) 감독 4) 출연정보 \n')
    if info_category == 'q':
        return 'q'
    elif info_category == 'c':
        return 'c'

    if info_category == '1':
        print('영화 정보를 입력해주세요')
        print('영화 이름, 감독 이름, 장르, 별점, 개봉일 순으로 입력해주세요')
        print('EX) 미나리, 정이삭, 드라마, 8.33, 2021-03-03')
        info_movie = input()
        info_movie = info_movie.split(',')
        info_movie = [x.strip() for x in info_movie]
        #설국열차, 봉준호, SF, 7.98, 2013-08-01
        if len(info_movie) == 5:
            info_movie[1] = convert_name_id(info_movie[1],3)
            query = '''
            insert movie values
            (null, '{}', '{}', '{}', '{}', '{}') '''.format(*info_movie)
            csr = wrapper_sql(query)
            if csr == -2 :
                print('입력 형식에 맞지 않습니다. 다시 입력해주세요.')
# 택시운전사, 장훈, 드라마, 9.28, 2017-08-02
        else:
            print('5개의 정보를 입력해야 합니다. 다시 진행해주세요.')

    if info_category == '2':
        info_actor = input('배우 이름을 입력해주세요\n') 
        query = "insert actor values (null,'{}')".format(info_actor)
        csr = wrapper_sql(query)
        if csr == -2:
                print('입력 형식에 맞지 않습니다. 다시 입력해주세요.')

    if info_category == '3':
        info_director = input('감독 이름을 입력해주세요\n')
        query = '''
        insert director values
        (null, '{}')'''.format(info_director)
        csr = wrapper_sql(query)
        if csr == -2:
                print('입력 형식에 맞지 않습니다. 다시 입력해주세요.')
    
    if info_category == '4':
        print('영화 이름, 배우 이름, 감독 이름을 입력해주세요')
        print('EX) 미나리, 윤여정, 정이삭')
        info_info = input()
        info_info = info_info.split(',')
        info_info = [x.strip() for x in info_info]
        if len(info_info)!=3:
            print('입력 형식에 맞지 않습니다. 다시 입력해주세요.')
            return -2
        info_info = [convert_name_id(info_info[i], i+1) for i in range(3)]
        query = "insert info values (null, {}, {}, {})".format(info_info[0], info_info[1], info_info[2])
        csr = wrapper_sql(query)
        if csr == -2:
                print('입력 형식에 맞지 않거나 존재하지 않는 영화, 배우, 감독 정보입니다.')
        #insert info values (info_info[0], info_info[1], info_info[2])
        #error trouble shooting pass
    return 0
# 모드 1을 입력 -> 영화이름을 받고 그걸 아이디로 반환.
# 모드 2를 입력 -> 배우이름을 받고 그걸 아이디로 반환.
# 모드 3을 입력 -> 감독이릉르 받고 그걸 아이디로 반환.
def convert_name_id(name, mode):
    result_id = 0
    if mode == 1:
        query = '''
        select movie_id 
        from movie 
        where movie_name = '{}' '''.format(name)
        result_id = wrapper_sql(query)

    elif mode ==2:
        query = '''
        select actor_id
        from actor
        where name ='{}' '''.format(name)
        result_id = wrapper_sql(query)

    else:
        query = '''
        select director_id
        from director
        where name ='{}' '''.format(name)
        result_id = wrapper_sql(query)
    
    try:
        result = (result_id[0])[0]
        return result
    except:
        return -1
        

sql_movie = 'SELECT * FROM movie'
sql_actor = 'SELECT * FROM actor'
sql_director = 'SELECT * FROM director'
sql_info = 'SELECT * FROM info'

def search_movie():
    print('어떤 카테고리로 조회하시길 원하시나요? 카테고리를 선택해주세요.')
    search_category = input('1) 영화 이름 키워드 검색 2) 배우 검색 3) 감독 검색 4) 장르 검색 5)별점순\n')
    if search_category.isdigit(): 
        search_category = int(search_category)
    elif search_category == 'q':
        return 'q'
    elif search_category == 'c':
        return 'c'
    if search_category == 1:
        movie_keyword = input('영화 이름을 입력해주세요 : \n')
        query= "select * from movie where movie_name like '%{}%' ".format(movie_keyword)
        csr = wrapper_sql(query)
        # columns_of_table('movie', 15)

        if not csr or csr == -2:
            print('입력하신 이름을 가진 영화 데이터를 찾을 수 없습니다.')
        else:
            print('{}\t{}\t{}\t{}\t{}'.format('영화 이름','감독 이름','장르','평점','개봉일'))
            for i in csr:
                query_director_name = "select name from director where director_id = {}".format(i[2])
                d_name = wrapper_sql(query_director_name)[0][0]
                print('{:<7}\t{:<7}\t{:7}\t{:<7}\t{}'.format(i[1], d_name, i[3], i[4], i[5]))
                # print('{:<10}{:<10}{:<10}{:<10}{}'.format(i[1], '봉준호', i[3], i[4], i[5]) )

            
    elif search_category==2:
        movie_actor=input('배우 이름을 입력해주세요: \n')
        query_actor_id = "select actor_id from actor where name like '%{}%' ".format(movie_actor)
        actor_id = (wrapper_sql(query_actor_id))
        if not actor_id or actor_id == -2:
            print('입력하신 이름을 가진 배우 데이터를 찾을 수 없습니다.')
        else:
            actor_id = actor_id[0][0]
            query_movie = '''
            select *
            from (
                select i.actor_id, i.movie_id, m.movie_name, i.director_id, m.genre_name, m.rate, m.released_date
                from info i
                join movie m
                using (movie_id)) movie_info
            where actor_id = {}'''.format(actor_id)
            movies = (wrapper_sql(query_movie))
            if not movies or movies == -2:
                print('입력하신 이름을 가진 배우 데이터를 찾을 수 없습니다.')
            else:
                print('{}\t{}\t{}\t{}\t{}'.format('영화 이름', '감독 이름', '장르', '별점', '개봉일'))
                for i in movies:
                    query_director_name = "select name from director where director_id = {}".format(i[3])
                    d_name = wrapper_sql(query_director_name)[0][0]
                    print('{:<7}\t{:<7}\t{:<}\t{:<7}\t{}'.format(i[2], d_name, i[4], i[5], i[6]))
                    
    elif search_category==3:
        movie_director=input('감독 이름을 입력해주세요: \n')
        query_director_id = "select director_id from director where name like '%{}%' ".format(movie_director)
        director_id = (wrapper_sql(query_director_id))
        if not director_id or director_id == -2:
            print('입력하신 이름을 가진 감독 데이터를 찾을 수 없습니다.')
        else:
            director_id = director_id[0][0]
            query_movie = '''
            select *
            from director d
            join movie m using(director_id)
            where d.director_id = {}'''.format(director_id)
            movies = (wrapper_sql(query_movie))
            if not movies or movies == -2:
                print('입력하신 이름을 가진 감독 데이터를 찾을 수 없습니다.')
            else:
                print('{}\t{}\t{}\t{}'.format('영화 이름', '장르', '별점', '개봉일'))
                for mv in movies:
                    print('{:<7}\t{}\t{:<7}\t{}'.format(mv[3], mv[4], mv[5], mv[6]))

    elif search_category==4:
        print('장르를 선택해서 입력해주세요: 드라마, SF, 로맨스, 코미디, 공포, 액션')
        movie_gerne=input('장르를 입력해주세요: \n')
        query="select * from movie where genre_name like '%{}%' ".format(movie_gerne)
        csr= wrapper_sql(query)
        if not csr or csr == -2:
            print('입력하신 장르 데이터를 찾을 수 없습니다.')
        else:
            print('{}\t{}\t{}\t{}'.format('영화 이름', '장르', '별점', '개봉일'))
            for i in csr:
                print('{:<7}\t{}\t{:<7}\t{}'.format(i[1], i[3], i[4], i[5]))
    elif search_category==5:
        print('별점을 순서대로 영화를 나열합니다.')
        query='select movie_name,rate from movie order by rate desc'
        csr= wrapper_sql(query)
        for i, movie in enumerate(csr):
            print(i+1, '위: ', movie[0],movie[1])
        # if csr ==-2

def columns_of_table(table_name, column_width):
    query = f'''
    select column_name
    from information_schema.columns
    where table_name = '{table_name}'
    '''
    csr = wrapper_sql(query)
    for col in csr:
        print('{:15}'.format(col[0]), end='')
    print()

print("안녕하세요. 영화 정보 조회 프로그램입니다.")
while True:
    notify_string = '영화 정보를 조회하시려면 1, 영화 정보를 입력하시려면 2를 입력해주세요.\n취소하시려면 c를 종료하시려면 q를 입력해주세요'
    print('-' * (len(notify_string) + 15))
    print(notify_string)

    user_mode = input('1) 조회 2) 입력 \n')
    if user_mode == '1' :
        user_cmd = search_movie()
        if user_cmd == 'q': break
        if user_cmd == 'c': continue

    elif user_mode == '2': 
        user_cmd = insert_movie_info()
        if user_cmd == 'q': break
        if user_cmd == 'c': continue

    elif user_mode == 'q': break 
    else :
        print('잘못된 입력입니다. 정확히 입력해주세요.')





        


안녕하세요. 영화 정보 조회 프로그램입니다.
-----------------------------------------------------------------------------------
영화 정보를 조회하시려면 1, 영화 정보를 입력하시려면 2를 입력해주세요.
취소하시려면 c를 종료하시려면 q를 입력해주세요
1) 조회 2) 입력 
2
영화 정보를 입력할 수 있습니다.
1) 영화 2) 배우 3) 감독 4) 출연정보 
1
영화 정보를 입력해주세요
영화 이름, 감독 이름, 장르, 별점, 개봉일 순으로 입력해주세요
EX) 미나리, 정이삭, 드라마, 8.33, 2021-03-03
국제수사, 김봉한, 액션, 7.22, 2020-09-29
-----------------------------------------------------------------------------------
영화 정보를 조회하시려면 1, 영화 정보를 입력하시려면 2를 입력해주세요.
취소하시려면 c를 종료하시려면 q를 입력해주세요
1) 조회 2) 입력 
2
영화 정보를 입력할 수 있습니다.
1) 영화 2) 배우 3) 감독 4) 출연정보 
2
배우 이름을 입력해주세요
곽도원
-----------------------------------------------------------------------------------
영화 정보를 조회하시려면 1, 영화 정보를 입력하시려면 2를 입력해주세요.
취소하시려면 c를 종료하시려면 q를 입력해주세요
1) 조회 2) 입력 
2
영화 정보를 입력할 수 있습니다.
1) 영화 2) 배우 3) 감독 4) 출연정보 
3
감독 이름을 입력해주세요
김봉한
-----------------------------------------------------------------------------------
영화 정보를 조회하시려면 1, 영화 정보를 입력하시려면 2를 입력해주세요.
취소하시려면 c를 종료하시려면 q를 입력해주세요
1) 조회 2) 입력 
2
영화 정